In [3]:
import pandas as pd
import json
import sys
sys.path.append("..")

from toolset import Toolset
import json

In [ ]:
# for each API, search the most similar APIs for it

data_dir = "output/instruction_data/"
retriever = ToolRetrieval(model_name_or_path="iic/nlp_gte_sentence-embedding_english-base", top_k=6)

toolset = Toolset()
api_docs = toolset.get_api_docs()

api_list = []
doc_text_list = []
for tn, api_docs in api_docs.items():
    for an, doc in api_docs.items():
        doc['name'] = tn + "##" + an
        doc_text = json.dumps(doc, ensure_ascii=False)
        api_list.append((tn, an))
        doc_text_list.append(doc_text)

retriever.construct(doc_text_list)

result_list = []
for idx, src_d in enumerate(doc_text_list):
    res = retriever.retrieve(src_d)
    result_list.append(res)
    print(f"{idx}/{len(doc_text_list)}")

api_to_simi_apis_map = {}
for idx, simi_docs in enumerate(result_list):
    api = api_list[idx]
    api_to_simi_apis_map[api] = list(simi_docs.keys())

api_to_simi_apis_map.items()
filter_map = {}
for api, simi_apis in api_to_simi_apis_map.items():
    tmp = f"{api[0]}##{api[1]}"
    simi_apis.remove(tmp)
    top5_simi_apis = simi_apis[:5]
    filter_map[tmp] = top5_simi_apis

with open(os.path.join(data_dir, "simi_apis.json"), "w") as file:
    cont = json.dumps(filter_map, indent=2, ensure_ascii=False)
    file.write(cont)

In [9]:
df

,Unnamed: 0,did,tool_name,api_name,simi_apis,instruction,type,label,explain
0,0,10,agent_equipments,get_current_time,agent_equipments##get_my_location\ncalendar##d...,What's the current local time and date right now?,2,1,NaN
1,1,11,agent_equipments,get_current_time,agent_equipments##get_my_location\ncalendar##d...,What day of the week is it today?,2,1,NaN
2,2,12,agent_equipments,get_current_time,agent_equipments##get_my_location\ncalendar##d...,"Can you tell me the exact time, date, and day ...",2,1,NaN
3,3,13,agent_equipments,get_current_time,agent_equipments##get_my_location\ncalendar##d...,What's the current time and day today?,2,1,NaN
4,4,14,agent_equipments,get_current_time,agent_equipments##get_my_location\ncalendar##d...,Can you provide me the current time along with...,2,1,NaN
...,...,...,...,...,...,...,...,...,...
418,418,654,world_time_by_api_ninjas,v1_worldtime,geocoding_by_api_ninjas##v1_reversegeocoding\n...,"What is the time in Tokyo, Japan at the moment?",2,1,NaN
419,419,655,world_time_by_api_ninjas,v1_worldtime,geocoding_by_api_ninjas##v1_reversegeocoding\n...,Please tell me the current time for the timezo...,2,1,NaN
420,420,656,world_time_by_api_ninjas,v1_worldtime,geocoding_by_api_ninjas##v1_reversegeocoding\n...,"Can you find out the time in Sydney, Australia...",2,1,NaN
421,421,658,world_time_by_api_ninjas,v1_worldtime,geocoding_by_api_ninjas##v1_reversegeocoding\n...,"How late is it in Moscow, Russia at this moment?",2,1,NaN


In [12]:
# produce data for task 2

type_ii_fn =  "../instruction_data/proc_instructions_type-ii.csv"
simi_api_fn = "../instruction_data/simi_apis.json"

df = pd.read_csv(type_ii_fn)

with open(simi_api_fn) as file:
    obj = json.loads(file.read())

data_list = []
for idx in df.index:
    row = df.loc[idx]
    _, did, tool_name, api_name, instruction, inst_t = row.tolist()
    simi_apis = obj[f"{tool_name}##{api_name}"]
    simi_apis_text = "\n".join(simi_apis)
    data_list.append((did, tool_name, api_name, simi_apis_text, instruction, inst_t))
out_df = pd.DataFrame(data=data_list, columns=["did", "tool_name", "api_name", "simi_apis", "instruction", "instruction_type"])
out_df.to_csv("output/eval_tasks/task2.csv")

In [9]:
out_df

,did,tool_name,api_name,simi_apis,instruction,type,label,explain
0,10,agent_equipments,get_current_time,agent_equipments##get_my_location\ncalendar##d...,What's the current local time and date right now?,2,1,NaN
1,11,agent_equipments,get_current_time,agent_equipments##get_my_location\ncalendar##d...,What day of the week is it today?,2,1,NaN
2,12,agent_equipments,get_current_time,agent_equipments##get_my_location\ncalendar##d...,"Can you tell me the exact time, date, and day ...",2,1,NaN
3,13,agent_equipments,get_current_time,agent_equipments##get_my_location\ncalendar##d...,What's the current time and day today?,2,1,NaN
4,14,agent_equipments,get_current_time,agent_equipments##get_my_location\ncalendar##d...,Can you provide me the current time along with...,2,1,NaN
...,...,...,...,...,...,...,...,...
418,654,world_time_by_api_ninjas,v1_worldtime,geocoding_by_api_ninjas##v1_reversegeocoding\n...,"What is the time in Tokyo, Japan at the moment?",2,1,NaN
419,655,world_time_by_api_ninjas,v1_worldtime,geocoding_by_api_ninjas##v1_reversegeocoding\n...,Please tell me the current time for the timezo...,2,1,NaN
420,656,world_time_by_api_ninjas,v1_worldtime,geocoding_by_api_ninjas##v1_reversegeocoding\n...,"Can you find out the time in Sydney, Australia...",2,1,NaN
421,658,world_time_by_api_ninjas,v1_worldtime,geocoding_by_api_ninjas##v1_reversegeocoding\n...,"How late is it in Moscow, Russia at this moment?",2,1,NaN


In [7]:
obj

{'weatherapi_com##realtime_weather_api': ['weatherapi_com##forecast_weather_api',
  'weatherapi_com##time_zone_api',
  'weatherapi_com##astronomy_api',
  'weatherapi_com##search_autocomplete_api',
  'weatherapi_com##ip_lookup_api'],
 'weatherapi_com##forecast_weather_api': ['weatherapi_com##astronomy_api',
  'weatherapi_com##realtime_weather_api',
  'weatherapi_com##time_zone_api',
  'weatherapi_com##search_autocomplete_api',
  'weatherapi_com##ip_lookup_api'],
 'weatherapi_com##ip_lookup_api': ['ip_geo_location##ip_lookup',
  'ip_geo_location##visitor_lookup',
  'weatherapi_com##realtime_weather_api',
  'weatherapi_com##forecast_weather_api',
  'weatherapi_com##astronomy_api'],
 'weatherapi_com##time_zone_api': ['weatherapi_com##search_autocomplete_api',
  'weatherapi_com##astronomy_api',
  'weatherapi_com##realtime_weather_api',
  'weatherapi_com##forecast_weather_api',
  'weatherapi_com##ip_lookup_api'],
 'weatherapi_com##astronomy_api': ['weatherapi_com##forecast_weather_api',
  'w